In [1]:
import pandas as pd

In [2]:
data=pd.read_csv(r'/Users/andyhsu/Desktop/statistic/ANOVA/consumption_data.csv',encoding='big5')

In [3]:
data

,地區,廣告,消費金額
0,南部,廣告1,20.83
1,南部,廣告1,21.45
2,中部,廣告1,27.09
3,南部,廣告1,14.09
4,南部,廣告1,31.23
...,...,...,...
355,南部,廣告3,9.69
356,北部,廣告3,10.29
357,南部,廣告3,25.52
358,北部,廣告3,19.26


In [4]:
alist = data[data ['廣告'] == '廣告1']['消費金額'].tolist()
blist = data[data ['廣告'] == '廣告2']['消費金額'].tolist()
clist = data[data ['廣告'] == '廣告3']['消費金額'].tolist()

In [5]:
print('alist:',alist)
print('blist:',blist)
print('clist:',clist)

alist: [20.83, 21.45, 27.09, 14.09, 31.23, 27.2, 26.65, 22.22, 21.68, 22.62, 12.89, 17.17, 15.86, 14.41, 14.88, 10.62, 16.81, 11.52, 15.84, 16.7, 18.3, 24.57, 21.74, 20.56, 17.35, 17.04, 20.2, 19.73, 24.23, 14.14, 18.52, 18.64, 15.61, 21.62, 13.22, 14.16, 14.65, 19.71, 17.84, 19.53, 15.18, 20.44, 26.66, 20.3, 29.63, 23.87, 14.36, 20.99, 15.19, 21.39, 21.43, 17.81, 9.36, 29.45, 12.69, 14.32, 9.79, 22.08, 14.63, 11.68, 7.98, 8.05, 8.19, 18.14, 28.34, 6.08, 13.13, 11.71, 17.69, 17.72, 7.44, 18.07, 26.62, 6.9, 11.44, 26.96, 17.9, 10.84, 15.88, 10.17, 14.99, 16.0, 18.35, 6.54, 6.92, 14.6, 24.66, 17.4, 17.72, 11.9, 16.81, 23.21, 18.96, 19.38, 3.58, 19.35, 12.63, 4.72, 12.43, 28.97, 12.52, 25.36, 8.41, 25.26, 17.77, 18.23, 8.97, 11.89, 14.03, 15.5, 17.62, 15.43, 16.82, 20.7, 15.34, 19.84, 13.75, 19.0, 22.23, 14.81]
blist: [20.23, 18.1, 12.34, 24.08, 22.39, 19.79, 19.39, 18.76, 21.6, 23.54, 24.05, 13.31, 12.49, 12.18, 18.48, 18.13, 25.57, 25.15, 20.68, 19.87, 8.12, 12.04, 24.12, 30.84, 19.44, 

# 事前驗證

In [6]:
import scipy.stats as st

In [7]:
print(st.shapiro(alist))#statisticm越接近ㄧ，表示與常態分佈越擬合
print(st.shapiro(blist))
print(st.shapiro(clist))

ShapiroResult(statistic=0.9915865659713745, pvalue=0.6819638013839722)
ShapiroResult(statistic=0.9929247498512268, pvalue=0.805463969707489)
ShapiroResult(statistic=0.9802003502845764, pvalue=0.07433854043483734)


In [8]:
print(st.levene(alist,blist,clist,center='mean'))

LeveneResult(statistic=0.6891243668404422, pvalue=0.5026819827621525)


In [9]:
st.f_oneway(alist, blist, clist)

F_onewayResult(statistic=2.363535251734209, pvalue=0.0955577857122512)

# 事後檢定

In [10]:
import pingouin
from pingouin import pairwise_tukey
m_comp = pairwise_tukey(data=data, dv='消費金額', between='廣告')

In [11]:
m_comp

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,廣告1,廣告2,17.135167,15.592417,1.54275,0.781281,1.974642,0.120126,0.254121
1,廣告1,廣告3,17.135167,15.748167,1.38700,0.781281,1.775290,0.179498,0.228466
2,廣告2,廣告3,15.592417,15.748167,-0.15575,0.781281,-0.199352,0.900000,-0.025655


In [12]:
table = m_comp.drop(columns = ['mean(A)', 'mean(B)', 'T', 'p-tukey', 'hedges'])
table

,A,B,diff,se
0,廣告1,廣告2,1.54275,0.781281
1,廣告1,廣告3,1.38700,0.781281
2,廣告2,廣告3,-0.15575,0.781281


# 新資料表

In [13]:
# 「A」欄反轉資料
add_A = table['B'].tolist()
# 「B」欄反轉資料
add_B = table['A'].tolist()
# 「diff」欄反轉資料
diff =  (table['diff'] - 2 * table['diff']).tolist()
# 「se」欄反轉資料
se = table['se'].tolist()
# 將反轉資料合併
table2 = pd.DataFrame(zip(add_A, add_B, diff, se), columns = ['A', 'B', 'diff', 'se'])

In [14]:
table2

,A,B,diff,se
0,廣告2,廣告1,-1.54275,0.781281
1,廣告3,廣告1,-1.38700,0.781281
2,廣告3,廣告2,0.15575,0.781281


In [15]:
new_table = pd.concat([table, table2], ignore_index=True)

In [16]:
new_table

,A,B,diff,se
0,廣告1,廣告2,1.54275,0.781281
1,廣告1,廣告3,1.38700,0.781281
2,廣告2,廣告3,-0.15575,0.781281
3,廣告2,廣告1,-1.54275,0.781281
4,廣告3,廣告1,-1.38700,0.781281
5,廣告3,廣告2,0.15575,0.781281


In [17]:
new_table['上界'] = new_table['diff'] + new_table['se']*1.96
new_table['下界'] = new_table['diff'] - new_table['se']*1.96

In [18]:
new_table

,A,B,diff,se,上界,下界
0,廣告1,廣告2,1.54275,0.781281,3.074061,0.011439
1,廣告1,廣告3,1.38700,0.781281,2.918311,-0.144311
2,廣告2,廣告3,-0.15575,0.781281,1.375561,-1.687061
3,廣告2,廣告1,-1.54275,0.781281,-0.011439,-3.074061
4,廣告3,廣告1,-1.38700,0.781281,0.144311,-2.918311
5,廣告3,廣告2,0.15575,0.781281,1.687061,-1.375561


In [26]:
import numpy as np
new_table['顯著']=np.NaN
for x in range(len(new_table)):
    if np.sign(new_table.iloc[x,4])==1.0:
        if np.sign(new_table.iloc[x,5])==1.0:
            new_table.iloc[x,-1]='Yes'
        else:
            new_table.iloc[x,-1]='No'
    else:
        if np.sign(new_table.iloc[x,4])!=1.0:
            new_table.iloc[x,-1]='Yes'
        else:
            new_table.iloc[x,-1]='No'

In [27]:
new_table

,A,B,diff,se,上界,下界,顯著
0,廣告1,廣告2,1.54275,0.781281,3.074061,0.011439,Yes
1,廣告1,廣告3,1.38700,0.781281,2.918311,-0.144311,No
2,廣告2,廣告3,-0.15575,0.781281,1.375561,-1.687061,No
3,廣告2,廣告1,-1.54275,0.781281,-0.011439,-3.074061,Yes
4,廣告3,廣告1,-1.38700,0.781281,0.144311,-2.918311,No
5,廣告3,廣告2,0.15575,0.781281,1.687061,-1.375561,No


In [31]:
import plotly.offline as py
import plotly.graph_objects as go
fig = go.Figure()
for i in range(0, new_table.shape[0]):
      if new_table.iloc[i,6] == 'Yes':
          color = 'firebrick'
          name = '顯著'
      else:
          color = 'green'
          name = '不顯著'
      fig.add_trace(go.Scatter(
          x = [new_table.iloc[i,5], new_table.iloc[i,2], new_table.iloc[i,4]],
          y = [new_table.iloc[i,0] + '-' + new_table.iloc[i,1], new_table.iloc[i,0] + '-' + new_table.iloc[i,1], new_table.iloc[i,0] + '-' + new_table.iloc[i,1]],
          mode = "lines+markers",
          textfont=dict(
family="sans serif",
              size=16,
              color=color),    
            line=dict(color=color, width=2),
          name = name,
          legendgroup = name,
          ))
    
fig.update_layout(
      title={
          'text': "<b>One-Way ANOVA 廣告效益分析</b>",
          'y':0.95,
          'x':0.5,
          'xanchor': 'center',},
      width=1000,
      height=800,
      boxmode='group',
      font=dict(
          family="Courier New, monospace",
          size=20,
          color="lightslategrey"
      )
      )

# 另存互動式網頁
py.plot(fig, filename='One-Way ANOVA 廣告效益分析', auto_open=True)

'One-Way ANOVA 廣告效益分析.html'